<h3>Импортируем библиотеки</h3>

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import json
from sklearn.naive_bayes import MultinomialNB
import pickle
import nltk
from sklearn import metrics
nltk.download('punkt')
from collections import defaultdict

[nltk_data] Downloading package punkt to /home/anna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<h3>Считываем данные</h3>

In [2]:
train_table = pd.read_json('train.json')
test_table = pd.read_json('test.json')

<h3>Строим модель</h3>

In [3]:
def ru_token(string):
    return [i.lower() for i in word_tokenize(string) if re.match(r'[\u0400-\u04ffа́]+$', i)]

# Разбиваем нашу строку на подстроки и возвращаем список слов, которые подходят под кириллицу

params = {}
params['tokenizer'] = ru_token
params['stop_words'] = stopwords.words('russian')
params['ngram_range'] = (1, 3)
params['min_df'] = 3

tfidf  = TfidfVectorizer(**params)

# Задаем параметры для tf–idf 

Разделить количество употреблений каждого слова в документе на общее количество слов в документе. Этот новый признак называется tf — Частота термина.

Следующее уточнение меры tf — это снижение веса слова, которое появляется во многих документах в корпусе, и отсюда является менее информативным, чем те, которые используются только в небольшой части корпуса. Примером низко ифнормативных слов могут служить служебные слова, артикли, предлоги, союзы и т.п.

Это снижение называется tf–idf, что значит “Term Frequency times Inverse Document Frequency” (обратная частота термина).

In [4]:
all_data = train_table.append(test_table, ignore_index=True)
tfidf.fit([row['text'].lower() for index, row in all_data.iterrows()])

/tmp/ipykernel_38132/3310056640.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = train_table.append(test_table, ignore_index=True)
/home/anna/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(min_df=3, ngram_range=(1, 3),
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...],
                tokenizer=<function ru_token at 0x7fa827dfe8b0>)

<h3>Разделяем данные</h3>

Разделим **train_table** на два словаря, один будет составлять 80% исходных данных, второй 20% данных. Таким образом, это будут обучающие и тестовые данные соотвественно для оценки точности нашей модели.

Первый словарь будет в ключах иметь **sentiment** в значениях list из **text**. Затем воспользуемся функцией, **train_test_split** которая "split arrays or matrices into random train and test subsets."

In [5]:
train1, train2, tmp = defaultdict(list), defaultdict(list), defaultdict(list)
for index, row in train_table.iterrows():
    tmp[row['sentiment']].append(row['text'].lower())
for element in tmp:
    train1[element], train2[element] = train_test_split(tmp[element], test_size=0.2, random_state=2022)

[Здесь](https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/) можно почить о различных техниках по выравниваю данных и для чего вообще это нужно. Мы будем использовать Random Over-Sampling.

In [6]:
def align(some_dict):
    rand = np.random.RandomState(2022)
    
    max_len = 0
    
    for key in some_dict:
        max_len = max(max_len, len(some_dict[key]))
    print('max_len: ' + str(max_len))
  
    tmp = defaultdict(list)
    for key in some_dict:
        if len(some_dict[key]) < max_len:
            
            _tmp = some_dict[key].copy()
            rand.shuffle(_tmp)
            
            tmp[key] = some_dict[key] * (max_len // len(some_dict[key])) + _tmp[:(max_len % len(some_dict[key]))]
            rand.shuffle(tmp[key])
        else:
            tmp[key] = some_dict[key]
    return tmp

In [7]:
train1 = align(train1)

max_len: 3227


А теперь сформируем два массива **train_x** с текстами, **train_y** с их оценками. Посортируем ключи, чтобы сначала шли все тексты одного ключа, потом второго, затем тертьего и обучимся.

In [8]:
train_x = []
for i in sorted(train1.keys()):
    for j in train1[i]:
        train_x.append(j)
        
train_y = []
for i in sorted(train1.keys()):
    for j in train1[i]:
        train_y.append(i)
        
softmax = MultinomialNB()
softmax.fit(tfidf.transform(train_x), train_y)

MultinomialNB()

А теперь аналогично сформируем два массива **test_x** с текстами, **true** с их оценками. Посортируем ключи, чтобы сначала шли все тексты одного ключа, потом второго, затем тертьего. Сделаем предсказания и оценим насколько точно мы их сделали.

In [9]:
test_x = []
for i in sorted(train2.keys()):
    for j in train2[i]:
        test_x.append(j)
        
true = []
for i in sorted(train2.keys()):
    for j in train2[i]:
        true.append(i)

In [10]:
predicted = softmax.predict(tfidf.transform(test_x))

In [11]:
accuracy_score(true, predicted)

0.6981246218995766

<h3>Отправляем результаты</h3>

In [12]:
sub_pred = softmax.predict(tfidf.transform([row['text'].lower() for index, row in test_table.iterrows()]))
sub_df = pd.DataFrame()
sub_df['id'] =  [row['id'] for index, row in test_table.iterrows()]
sub_df['sentiment'] = sub_pred

sub_df.to_csv('submission.csv', index=False)

<h3>Сохраняем модель для бота</h3>

In [13]:
with open('NLP2.pkl', 'wb+') as f:
    pickle.dump((tfidf, softmax), f)

*Обучащие и тестовые данные взяты [отсюда](https://www.kaggle.com/competitions/sentiment-analysis-in-russian/overview)*.

*Файл с моделью вот [тут](https://disk.yandex.ru/d/KQPaKH7sBw0hvA)*